In [1]:
from numpy import expand_dims
from numpy import mean
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras import backend
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.initializers import RandomNormal
from keras.constraints import Constraint
from matplotlib import pyplot
 
    
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
# load all necessary libraries
import pandas as pd
import numpy as np # linear algebra

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.metrics import confusion_matrix as sk_confusion_matrix
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt


from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection  import train_test_split
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer


from gensim.models.word2vec import Word2Vec
from gensim import models
from gensim.models import KeyedVectors


from sklearn.linear_model import LogisticRegression
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.fashion_mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D,Lambda,BatchNormalization,Activation
from keras.layers import Conv1D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
import keras.backend as K

import tensorflow as tf
from keras.preprocessing import sequence

from keras.models import load_model
from numpy.random import randn
from matplotlib import pyplot
from keras.layers import Input,MaxPool1D
from keras.models import Model


Using TensorFlow backend.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/mnt/disks/user/anaconda3/lib/python3.6/site-pa

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
## initialise the inbuilt Stemmer and the Lemmatizer
stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

def preprocess(document, stem=True):
    'changes document to lower case, removes stopwords and lemmatizes/stems the remainder of the sentence'

    # change sentence to lower case
    document = document.lower()

    # tokenize into words
    words = word_tokenize(document)

    # remove stop words
    words = [word for word in words if word not in stopwords.words("english")]

    if stem:
        words = [stemmer.stem(word) for word in words]
    else:
        words = [wordnet_lemmatizer.lemmatize(word, pos='v') for word in words]

    # join words to make sentence
    document = " ".join(words)

    return document

The differences in implementation for the WGAN are as follows:

Use a linear activation function in the output layer of the critic model (instead of sigmoid).
Use -1 labels for real data and 1 labels for fake data (instead of 1 and 0).
Use Wasserstein loss to train the critic and generator models.
Constrain critic model weights to a limited range after each mini batch update (e.g. [-0.01,0.01]).
Update the critic model more times than the generator each iteration (e.g. 5).
Use the RMSProp version of gradient descent with a small learning rate and no momentum (e.g. 0.00005).

In [3]:
seq_len = max_len = 1024 #512
b_size = 128
n_chan = 10
f_size = 3 # filter size



data = pd.read_csv("./../deceptive-opinion.csv")
data = data.loc[:,['text','deceptive']]

# stem messages
#messages = [preprocess(message, stem=True) for message in data.text]
data.text = data.text.apply(lambda message : preprocess(message, stem=False))
data['deceptive'] = data.deceptive.map({'truthful':-1, 'deceptive':1})

X = data.text
y = data.deceptive

df_train, df_test= train_test_split(data,  test_size = 0.20,random_state=1)
df_truthful = df_train.loc[df_train.deceptive == -1,:]
X_train = df_truthful.text
X_test=df_test.text
y_train= df_truthful.deceptive
y_test=df_test.deceptive

print('Training set size : ', (X_train.shape[0]))
print('Test set size : ', (X_test.shape[0]))

bow_model = CountVectorizer(stop_words='english')#,max_features=max_len)


X_train_bow = bow_model.fit_transform(X_train)
X_test_bow = bow_model.transform(X_test)


Training set size :  649
Test set size :  320


In [4]:
X_train_bow.shape

(649, 5184)

In [5]:
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(X_train_bow.toarray(), maxlen=max_len)
x_test = sequence.pad_sequences(X_test_bow.toarray(), maxlen=max_len)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad sequences (samples x time)
x_train shape: (649, 1024)
x_test shape: (320, 1024)


In [6]:
# clip model weights to a given hypercube
class ClipConstraint(Constraint):
    # set clip value when initialized
    def __init__(self, clip_value):
        self.clip_value = clip_value

    # clip model weights to hypercube
    def __call__(self, weights):
        return backend.clip(weights, -self.clip_value, self.clip_value)

    # get the config
    def get_config(self):
        return {'clip_value': self.clip_value}
 


In [7]:
# calculate wasserstein loss
def wasserstein_loss(y_true, y_pred):
    return backend.mean(y_true * y_pred)
 
# define the standalone critic model
def define_critic(in_shape=(seq_len,1)):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # weight constraint
    const = ClipConstraint(0.01)


    model = Sequential()
    model.add(Conv1D(n_chan,f_size,activation='relu', padding='same',kernel_initializer=init, kernel_constraint=const,input_shape = (seq_len,1)))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv1D(n_chan,f_size,activation='relu',padding='same', kernel_initializer=init, kernel_constraint=const))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    # scoring, linear activation
    model.add(Flatten())
    model.add(Dense(1))
    # compile model
    opt = RMSprop(lr=0.00005)
    model.compile(loss=wasserstein_loss, optimizer=opt,  metrics=['accuracy'])
    #model.compile(optimizer='adam', loss='binary_crossentropy',  metrics=['accuracy'])
    return model



In [8]:
 
# define the standalone generator model
def define_generator(latent_dim):
    def Conv1DTranspose(inp,nf,ks,s=2,p='same'):
        x1 = Lambda(lambda x : K.expand_dims(x,axis=2))(inp)
        x2 = Conv2DTranspose(filters=nf,kernel_size=(ks,1),strides=(s,1),padding=p)(x1)
        return Lambda(lambda x :K.squeeze(x,axis=2))(x2)

    G = Sequential()
    G.add(Dense(int(seq_len/8)*n_chan,input_shape=(latent_dim,)))
    G.add(Reshape((int(seq_len/8),n_chan)))
    G.add(BatchNormalization(momentum= 0.8,epsilon=1.e-5))
    for i in range(0,2):
        G.add(Lambda(lambda x : Conv1DTranspose(x,n_chan,f_size)))
        G.add(BatchNormalization(momentum= 0.8,epsilon=1.e-5))

    G.add(Lambda(lambda x : Conv1DTranspose(x,1,3)))
    #G.add(Activation('sigmoid'))
    G.add(Activation('tanh'))

    G.summary()
    return G



In [9]:

# define the combined generator and critic model, for updating the generator
def define_gan(generator, critic):
    # make weights in the critic not trainable
    for layer in critic.layers:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(generator)
    # add the critic
    model.add(critic)
    # compile model
    opt = RMSprop(lr=0.00005)
    model.compile(loss=wasserstein_loss, optimizer=opt)
    return model
 


In [10]:
# load data
def load_real_samples():
    # load dataset
    (trainX, _), (_, _) = (x_train,y_train),(x_test,y_test)#load_data()
    return trainX 

# select real samples
def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    # select data
    X = dataset[ix]
    # generate class labels, -1 for 'real'
    y = -ones((n_samples, 1))
    return X, y
 
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input
 
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = generator.predict(x_input)
    # create class labels with 1.0 for 'fake'
    y = ones((n_samples, 1))
    return X, y
 

# create a line plot of loss for the gan and save to file
def plot_history(d1_hist, d2_hist, g_hist):
    # plot history
    pyplot.title("WGAN+BOW")
    pyplot.plot(d1_hist, label='crit_real')
    pyplot.plot(d2_hist, label='crit_fake')
    pyplot.plot(g_hist, label='gen')
    pyplot.legend()
    pyplot.savefig('WGAN_BOW_line_plot_loss.png')
    pyplot.close()
 


In [11]:
# train the generator and critic
def train(g_model, c_model, gan_model, dataset, latent_dim, n_epochs=20, n_batch=64, n_critic=5):
    # calculate the number of batches per training epoch
    bat_per_epo = int(dataset.shape[0] / n_batch)
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    # calculate the size of half a batch of samples
    half_batch = int(n_batch / 2)
    # lists for keeping track of loss
    c1_hist, c2_hist, g_hist = list(), list(), list()
    # manually enumerate epochs
    for i in range(n_steps):
        # update the critic more than the generator
        c1_tmp, c2_tmp = list(), list()
        for _ in range(n_critic):
            # get randomly selected 'real' samples
            X_real, y_real = generate_real_samples(dataset, half_batch)

            # update critic model weights
            c_loss1 = c_model.train_on_batch(X_real.reshape(X_real.shape[0],X_real.shape[1],1), y_real)
            c1_tmp.append(c_loss1[0])
            # generate 'fake' examples
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)


            # update critic model weights
            c_loss2 = c_model.train_on_batch(X_fake, y_fake)
        
            #print(c_loss1,c_loss2)
            c2_tmp.append(c_loss2[0])
        # store critic loss
        c1_hist.append(mean(c1_tmp))
        c2_hist.append(mean(c2_tmp))
        # prepare points in latent space as input for the generator
        X_gan = generate_latent_points(latent_dim, n_batch)
        # create inverted labels for the fake samples
        y_gan = -ones((n_batch, 1))
        # update the generator via the critic's error
        g_loss = gan_model.train_on_batch(X_gan, y_gan)
        g_hist.append(g_loss)
        # summarize loss on this batch
        print('>%d, c1=%.3f, c2=%.3f g=%.3f' % (i+1, c1_hist[-1], c2_hist[-1], g_loss))

    # line plots of loss
    plot_history(c1_hist, c2_hist, g_hist)
    
    return c_model

# size of the latent space
latent_dim = 50
# create the critic
critic = define_critic()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, critic)
# load data
dataset = load_real_samples()
print(dataset.shape)
# train model
c_model = train(generator, critic, gan_model, dataset, latent_dim)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 1280)              65280     
_________________________________________________________________
reshape_1 (Reshape)          (None, 128, 10)           0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 128, 10)           40        
_________________________________________________________________
lambda_1 (Lambda)            (None, 256, 10)           0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 256, 10)           40        
_________________________________________________________________
lambda_6 (Lambda)            (None, 512, 10)           0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 512, 10)           40        
__________

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>1, c1=-0.064, c2=-0.009 g=0.066
>2, c1=-0.051, c2=-0.153 g=0.206
>3, c1=-0.065, c2=-0.291 g=0.361
>4, c1=-0.073, c2=-0.453 g=0.555
>5, c1=-0.081, c2=-0.659 g=0.776
>6, c1=-0.081, c2=-0.849 g=1.014
>7, c1=-0.091, c2=-1.098 g=1.223
>8, c1=-0.081, c2=-1.340 g=1.536
>9, c1=-0.084, c2=-1.629 g=1.875
>10, c1=-0.118, c2=-1.932 g=2.142
>11, c1=-0.081, c2=-2.298 g=2.537
>12, c1=-0.105, c2=-2.601 g=2.988
>13, c1=-0.071, c2=-2.971 g=3.399
>14, c1=-0.073, c2=-3.384 g=3.836
>15, c1=-0.071, c2=-3.796 g=4.304
>16, c1=-0.102, c2=-4.252 g=4.833
>17, c1=-0.048, c2=-4.712 g=5.351
>18, c1=-0.076, c2=-5.181 g=5.841
>19, c1=-0.035, c2=-5.636 g=6.467
>20, c1=-0.025, c2=-6.177 g=7.016
>21, c1=-0.046, c2=-6.724 g=7.617
>22, c1=-0.066, c2=-7.257 g=8.269
>23, c1=0.001, c2=-7.829 g=8.846
>24, c1=0.002, c2=-8.437 g=9.572
>25, c1=-0.012, c2=-9.093 g=10.301
>26, c1=-0.046, c2=-9.677 g=11.012
>27, c1=0.029, c2=-10.387 g=11.663
>28, c1=0.054, c2=-11.024 g=12.343
>29, c1=0.055, c2=-11.673 g=13.160
>30, c1=0.088, c2=-1

In [12]:
# First, the loss of the critic and generator models is reported to the console each iteration of the training loop.
# Specifically, c1 is the loss of the critic on real examples, c2 is the loss of the critic in generated samples,
# and g is the loss of the generator trained via the critic.

# The c1 scores are inverted as part of the loss function; this means if they are reported as negative,
# then they are really positive, and if they are reported as positive, they are really negative.
# The sign of the c2 scores is unchanged.